 # Arreglos y operaciones vectoriales con [NumPy](http://www.numpy.org/)


NumPy es un paquete de computación científica con Python que provee:

- Un objecto contenedor muy versatil: arreglo N-dimensional `ndarray`
- Funciones capaces de hacer *broadcasting*
- Módulos para algebra lineal, Transformada de Fourier, generación de número aleatorios, entre otros
- Herramientas para integrar código C/C++


**Instalación**
    
Con nuestro ambiente conda activado:

    conda install numpy
    
Esto instalará numpy y las librerías de bajo nivel BLAS y MKL

Luego importamos usando

In [ ]:
import numpy as np
print("Version: ", np.__version__)

## Objeto ndarray (alias array)

Una lista `lista` de Python es un tipo de arreglo donde cada elemento puede ser de tipo diferente

En general es muy ineficiente hacer cálculos numéricos usando listas

Como reemplazo usaremos el objeto `ndarray` de NumPy, que corresponde a un **arreglo n-dimensional de tipo fijo**

A diferencia de una lista, las operaciones matemáticas y reducciones sobre ndarray son eficientes

Podemos crear un ndarray a partir de 

- una lista o tupla usando `np.array`
- un fichero, por ejemplo usando `np.genfromtxt`
- funciones generadoras de NumPy, por ejemplo `np.linspace`, `np.zeros`, etc

### Ndarray a partir de listas y atributos básicos

In [ ]:
# Supongamos que tenemos la siguiente lista de listas
L = [[0, 1, 2], [3, 4, 5]]
print("L es un", type(L))
# Podemos transformarla a ndarray con
A = np.array(L)
print("A es un", type(A))
display(A)

Los tipos de dato estándar de NumPy son:

- Enteros: int8, int16, int32, int64
- Enteros sin signo: uint8, uint16, uint32, uint64
- Flotantes (reales): float16, float32, float64, float128
- Números complejos: complex64, complex128, complex256
- Booleanos: Bool

Podemos forzar el tipo usando el argumento `dtype`

In [ ]:
display(np.array(L, dtype=np.int16))
display(np.array(L, dtype=np.float32))

Además el atributo `dtype` nos permite ver el tipo de un arreglo NumPy

In [ ]:
x = np.array([1. + 2j])
print("x es de tipo", x.dtype)


- El atributo `ndim` es un entero que nos indica el número de dimensiones o ejes del arreglo
- El atributo `shape` es una tupla de entero que nos indica el tamaño del arreglo en cada una de sus dimensiones

Por ejemplo:

In [ ]:
display(A)
print("A tiene ", A.ndim, "dimensiones/ejes")
display(A.shape)
print("El eje 0 tiene largo", A.shape[0])
print("El eje 1 tiene largo", A.shape[1])

**Ordenamiento en memoria de ndarray**

Por defecto un ndarray multidimensional se ordena en memoria siguiente un formato `row-major` similar a la convención usada en `C`. Alternativamente un arreglo se puede guardar en formato `column-major` similar a la convención usada en `Fortran`. La siguiente figura muestra la diferencia

<img src="img/rowcolmajor.png" width="600">

El atributo `order` nos permite seleccionar el ordenamiento al momento de crear el arreglo

También se puede verificar el ordenamiento de un ndarray leyendo su atributo `flags`

- Si tiene el flag `C_CONTIGUOUS` verdadero entonces es `row major`
- Si tiene el flag `F_CONTIGUOUS` verdadero entonces es `column major`


In [ ]:
A = np.array(L)
display(A)
display(A.flags['C_CONTIGUOUS']) # Por defecto es True

A = np.array(L, order='F')
display(A)
display(A.flags['C_CONTIGUOUS'])

El diccionario `flags` contiene los siguientes valores:

In [ ]:
A.flags

### Crear un ndarray a partir de ficheros

Cuando se trabajo con tablas de datos es muy común que estas se distribuyan como fichero en formato csv

Un fichero csv es tipicamente un texto plano con valores separados por comas como el que se muestra a continuación

In [ ]:
# Por ejemplo:
!cat example.csv

El proceso de transformar este archivo en texto plano a una estructura de datos numérica se llama *parseo*

NumPy tiene algunas [operaciones de IO para conectar con ficheros e importar datos](https://numpy.org/devdocs/reference/routines.io.html). Por ejemplo para leer un archivo en formato CSV podemos usar  [genfromtxt](https://numpy.org/devdocs/user/basics.io.genfromtxt.html) especificando adecuadamente el caracter deliminator y opcionalmente el tipo


In [ ]:
data = np.genfromtxt('example.csv', delimiter=',')
data

Por defecto se importo como un arreglo de números flotantes. Podemos evitar lo anterior y forzar el tipo entero con

In [ ]:
data = np.genfromtxt('example.csv', 
                     delimiter=',', # Separador entre caracteres
                     dtype=int
                    )
data

```{note}
No se darán más detalles pues más adelante veremos la librería `pandas` que provee funciones mucho más poderosas y flexibles para hacer *parsing* de datos 
```

### Funciones generadoras de arreglos

Se pueden crear arreglos directamente desde Numpy

Algunos ejemplos útiles

In [ ]:
display(np.zeros(shape=(3, 3), dtype=np.int))  # Lleno de ceros
display(np.ones(shape=(3, 3), dtype=np.float32))  # Lleno de unos
display(np.full(shape=(3, 3), fill_value=np.pi))  # Lleno de PI
display(np.eye(3))  # Matriz identidad
display(np.random.randn(3, 3))  # Matriz aleatoria con distribución N(0, 1)

También existen versiones de estas funciones que copian el tamaño de otro ndarray

In [ ]:
display(np.zeros_like(A))

Las siguientes funciones son muy útiles cuando necesitamos crear un rango lineal o logarítmico usando

Especificamos el inicio, el fin y el paso o cantidad de elementos

In [ ]:
display(np.arange(start=0, stop=5, step=0.5))  # paso
display(np.linspace(start=0, stop=10, num=11)) # cantidad de elementos
display(np.logspace(start=-1, stop=1, num=11))

## Manipulación de matrices y vectores

Es usual que antes de operar un ndarray necesitemos cambiar su tamaño o número de dimensiones

Algunas operaciones típicas para modificar la forma de un arreglo son: `reshape`, `tile`, `repeat`, `ravel` y `transpose`

`reshape` reorganice las dimensiones de un arreglo pero debe preservar el tamaño

In [ ]:
A = np.arange(6)
display(A)
# Convierte 6 a 3x2
display(np.reshape(A, (3, 2)))  
# Convierte 6 a 2x3
display(np.reshape(A, (2, 3)))

`tile` repite el arreglo en una dirección dada

In [ ]:
# Repite 4 veces en la dirección de las filas
display(np.tile(A, (4, 1)))
# Repite 2 veces en la dirección de las columnas
display(np.tile(A, (1, 2)))

`repeat` repite cada elemento en una dirección o eje dado

In [ ]:
# Cada elemento aparece dos veces (duplicación)
display(np.repeat(A, 2))
# Cada elemento/fila aparece dos veces en la dirección de las filas
display(np.repeat(A.reshape(3, 2), 2, axis=0))
# Cada elemento aparece dos veces en la dirección de las columnas
display(np.repeat(A.reshape(3, 2), 2, axis=1))

`ravel` es una función que aplana el ndarray y retorna un arreglo de una dimensión

In [ ]:
# Convierte una matriz de 5x5 en un arreglo de 25
display(np.ravel(np.zeros(shape=(5, 5))))

`transpose` puede utilizarse para intercambiar la posición de los ejes/dimensiones de un ndarray

Tiene el mismo significado de la trasposición matricial

<img src="img/transpose.jpg" width="400">

Por ejemplo:

In [ ]:
A = np.arange(9).reshape(3, 3)
display(A)
display(np.transpose(A)) # Equivalente a A.transpose() o A.T

`transpose` puede usarse en ndarrays de cualquier dimensionalidad

Podemos usar el argumento `axes` para especificar cuales dimensiones se van a intercambiar

(Opcionalmente podemos usar la función `np.swapaxes()` para obtener le mismo efecto)

In [ ]:
A = np.arange(8).reshape(2, 2, 2)
display(A)
display(np.transpose(A, axes=(0, 2, 1)))  

**Agregar dimensiones a un arreglo**

En algunas ocasiones nos interesará extender un arreglo, agregándole dimensiones

Consideremos el siguiente arreglo unidimensional

In [ ]:
A = np.array([0, 1, 2, 3, 4]) 
print("Dimensión:", A.ndim, ", Tamaño: ", A.shape)

Como vimos antes podemos agregar una dimensión usando `reshape`, pero otra forma más simple es usando `np.newaxis` (un alias de `None`)

Si queremos agregarle una dimensión al arreglo anterior podemos hacerlo a la derecha o a la izquierda

Por ejemplo agregarle una dimensión a la derecha creará una matriz de $N\times1$ o vector columna

In [ ]:
Amod = A[:, np.newaxis]
display(Amod)
print("Dimensión:", Amod.ndim, ", Tamaño: ", Amod.shape)

Mientras que agregarle una dimensión a la izquierda creará una matriz de $1\times N$ o vector fila

In [ ]:
Amod = A[np.newaxis, :]
display(Amod)
print("Dimensión:", Amod.ndim, ", Tamaño: ", Amod.shape)

Algunas operaciones útiles para combinar arreglos son: `concatenate`, `vstack`, `hstack`

`concatenate` es más general que las dos últimas

In [ ]:
A = np.arange(6).reshape(1, 6) # 1,2,3,4,5,6
B = np.ones(shape=(1,6)) #1,1,1,1,1,1

# Combinar en eje filas
display(np.concatenate((A, B), axis=0)) 
# Combinar en eje columnas
display(np.concatenate((A, B), axis=1))
# Combina siempre en fila
display(np.vstack((A, B)))
# Combina siempre en columna
display(np.hstack((A, B)))

Finalmente resaltar que existen funciones para agregar o quitar elementos: `append`, `insert`, `delete`

In [ ]:
A = np.array([1., 2., 3.])
display(A)
# Agrega un elemento al final
display(np.append(A, 4))
# Agrega un elemento en la posición indicada
display(np.insert(A, 2, values=0.))
# Elimina el elemento en la posición indicada (retorna un nuevo arreglo)
display(np.delete(A, 2))

## Indexación y *slicing*

Al igual que otros contenedores de Python los ndarray soportan *slicing*

> Slicing es crear una arreglo a partir de una indexación sobre otro arreglo

Sea por por ejemplo:

In [ ]:
L = [[0, 1, 2], [3, 4, 5]]
A = np.array(L)
display(A)

Para acceder al elemento en la segunda fila y primera columna usaríamos

In [ ]:
display(L[1][0], # Con la lista
        A[1, 0]) # Con el ndarray

El ndarray nos da mucha flexibilidad para hacer slicing

In [ ]:
display(A[:, 1])  # Retorna la segunda columna
display(A[0, :]) # Retorna la primera fila 
display(A[1, ::2]) # Retorna los elementos de la primera fila y columnas pares
display(A[-1, -2]) # Retorna los elementos de la ultima fila y penultima columna

También podemos usar un arreglo de enteros para indexar otro arreglo

Esto se llama *fancy indexing*

In [ ]:
ix = np.array([0, 0, 1])
iy = np.array([0, 1, 1])
display(A[ix, iy]) # Elementos [0,0], [0,1] y [1,1]

Con un slice podemos escribir directamente en esos valores y modificarlos:

In [ ]:
A[ix, iy] = 10 
display(A)

También podemos indexar usando un arreglo de booleanos

In [ ]:
A = np.array([0, 2, 1, 3, 4])
B = np.array([True, False, False, True, True])
display(A[B])

La siguiente figura muestra de forma esquemática varios ejemplos de slices

<img src="img/slicing.png" width="700">

```{note}
Algunas operaciones sobre arreglos no hacen copias (usan referencias)
```

En particular cuando hacemos un slice, estamos modificando el arreglo original

In [ ]:
A = np.arange(100).reshape(10, 10)
B = A
B is A

Si modifico A se ve reflejado en B

In [ ]:
A[:5, :5] = 100
display(B)

Modificaciones en subarreglos (vistas) también son referenciadas

In [ ]:
A = np.arange(100).reshape(10, 10)
B = A[:5, :5]
display(B is A)
B[:, :] = 100
display(A)

Si queremos evitar este comportamiento se puede forzar la creación de una copia con el método `copy()`

In [ ]:
B = A.copy()
A[0, 0] = 0
display(B[0])

## Operaciones sobre ndarray

### Operaciones aritméticas y *Broadcasting* 

Los ndarray soportan las operaciones aritméticas básicas

- Suma:  +, +=
- Resta: -, -=
- Multiplicación:  *,*= 
- División: /, /=
- División entera: //, //=
- Exponenciación: ** , **=

Estas operaciones tienen un comportamiento element-wise (elemento a elemento), es decir

$$
\pmatrix{0 & 1 \\2 & 3 } \cdot \pmatrix{1 & 5 \\2 & 2 } = \pmatrix{0 & 5 \\4 & 6 }
$$

Note que no corresponde a la multiplicación usual de matrices

Veamos algunos ejemplos:

In [ ]:
N = 3
A = np.eye(N)
B = np.ones(shape=(N, N))
display(A)
display(B)
display(A + B)
display(A*B)  

Cuando los términos no son del mismo tamaño se hace un *broadcast*

Por ejemplo si operamos una constante con un arreglo, la constante se opera con cada elemento del arreglo

In [ ]:
A - 1

**Reglas de *broadcasting* en Numpy**

1. Si dos arreglos son de dimensiones distintas la dimensión del más pequeño se agranda con "1"s **por la izquierda**
1. Si dos arreglos tienen tamaños ditintos, el que tiene tamaño "1" se estira en dicha dimensión
1. Si en cualquier dimensión los tamaños son distintos y ninguno es igual a "1" ocurre un error


<img src="img/broadcast.png" width="550">

Imagen tomada del [Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/06.00-figure-code.html#Broadcasting)

Observe los siguientes ejemplos y reflexione sobre las reglas de broadcast que se está aplicando en cada caso

In [ ]:
C = np.arange(N)
B = np.ones(shape=(N, N))
display(C)
display(B)
display(B + C)
display(B + C.reshape(-1, 1))

In [ ]:
display(C.reshape(1, N))
display(C.reshape(N, 1))
display(C.reshape(1, N) + C.reshape(N, 1))

### Operaciones matriciales

Antes dijimos que la multiplicación con `*` se realiza elemento a elemento

Para realizar una multiplicación matricial propiamente tal podemos usar `dot` o el operador `@`

In [ ]:
A = np.arange(4).reshape(2, 2)
B = np.arange(4)[::-1].reshape(2, 2)
display(A, B)  

Note la diferencia:

In [ ]:
display(A*B)  
display(np.dot(A, B))

Otras operaciones matricionales útiles son:

- `np.inner` que calcula el producto escalar o producto interno
- `np.outer` que calcula el producto externo
- `np.cross` que calcula  cruz

In [ ]:
display(np.inner(A, B))
display(np.outer(A, B))

El módulo  [`linalg`](https://numpy.org/doc/stable/reference/routines.linalg.html) de NumPy contiene muchas más funciones de álgebra lineal que nos permiten 

- Calcular matriz inversa, determinantes y trazas
- Resolver sistemas lineales
- Factorizar matrices
- Calcular valores y vectores propios

entre otros. Este módulo será estudiando en detalle en lecciones más avanzadas

### Operaciones de reducción

Llamamos **reducción** a una operación que **agrega** los valores de un arreglo entregando un único valor como respuesta

La reducción más básica es la **suma agregada**

$$
[0, 1, 2, 4, 3] \rightarrow 0 + 1 + 2 + 4 + 3 = 10 
$$

> Las operaciones de reducción se usan ampliamente para resumir datos y hacer estadística


Algunas de las reducciones disponibles en NumPy son:
- `sum`, `prod`
- `amax`, `amin`, `argmax`, `argmin`
- `mean`, `std`, `var`, `percentile`, `median`
- `cumsum`, `cumprod`

Diferencia entre sumar en el eje de filas, columnas y suma total:

In [ ]:
A = np.tile(np.arange(3), (3, 1))
display(A)
display(np.sum(A, axis=0))
display(np.sum(A, axis=1))
display(np.sum(A))

Encontrar el valor y posición del máximo en un arreglo es también un tipo de reducción

In [ ]:
A = np.random.randn(3, 3)
display(A)
display(np.amax(A, axis=0))
display(np.argmax(A, axis=0))

Las operaciones de reducción de NumPy son altamente eficientes

Hagamos una pequeña prueba de desempeño sumando  un vector

Usaremos la magia de IPython `@timeit` que nos permite medir tiempo de cómputo

In [ ]:
A = np.arange(100000)

def suma_loop(arreglo):
    suma = 0.
    for elemento in arreglo:
        suma += elemento
    return suma

L = list(A)
%timeit -n10 suma_loop(A)
%timeit -n10 suma_loop(L)
%timeit -n10 sum(A)
%timeit -n10 sum(L)
%timeit -n10 np.sum(L)
%timeit -n10 np.sum(A)

display(np.sum(A))
display(sum(L))

Muchas de estas funciones están implementadas como métodos de la clase arreglo, por ejemplo

In [ ]:
display(A.sum())
display(A.max(axis=0))
display(A.min())

También cuentan con versiones seguras contra NaNs

In [ ]:
A = np.array([1., 10., 2., np.nan])
display(np.sum(A))
display(np.nansum(A))

**Nota:** Si queremos encontrar los NaN en un arreglo podemos usar `isnan` 

In [ ]:
np.isnan(A)

### Operaciones vectorizadas

Son funciones que operan de forma *element-wise* o elemento a elemento

Ya vimos las operaciones aritméticas elemento a elemento pero existen muchas más

Por ejemplo para calcular el valor absoluto de los elementos de un arreglo

In [ ]:
A = np.random.randn(3, 3)
display(A)
np.absolute(A) # Equivalente a np.abs(A)

Exponenciar un arreglo

In [ ]:
x = np.arange(10)
display(np.power(x, 2)) # Equivalente a x**2
display(np.sqrt(x))

Calcular funciones exponencial, logaritmo y trigonométricas  a partir de una arreglo

In [ ]:
x = np.array([0.1, 1., 10.0])
display(np.log(x)) # También está log2, log10
display(np.exp(x)) 
display(np.sin(x)) # También está arcsin, sinh
display(np.cos(x)) # También está arccos, cosh
display(np.tan(x)) #También está arctan, tanh

Otras funciones útiles: 

- Para obtener el signo de cada elemento de un arreglo: `sign`
- Para obtener un arreglo de 1 dividido los elementos del mismo: `reciprocal`
- Para redondear hacia abajo o hacia arriba: `round`, `floor` y `ciel`
- Para obtener la parte real o imaginaria de un número complejo: `real`, `imag`
- O el conjugado de un número complejo: `conj`, 

### Operaciones booleanas 

NumPy soporta operaciones booleanas sobre ndarray

In [ ]:
A = np.arange(6).reshape(2, 3)
display(A)
display(A == 4)
display(np.equal(A, 4))

Como vimos antes podemos crear una máscara booleana para indexar un arreglo

In [ ]:
mask = ~(A % 2 == 0) & (A > 2)
display(mask)
display(A[mask])

La función `where` sirve para recuperar el índice de los elementos que cumplen una cierta condición

In [ ]:
(ixs, iys) = np.where(~(A % 2 == 0) & (A > 2))

for i, j in zip(ixs, iys):
    display("Fila {0} Columna {1} Valor {2}".format(i, j, A[i, j]))

Funciones `any` y `all`

In [ ]:
x = np.random.randn(3, 3)
display(x)
b = (x > 0) & (x**2 > 0.5)
display(b)

display(np.any(b))
display(np.all(b))

### Operaciónes de conjutos

Operaciones de tipo union, intersección y diferencia entre arreglos 1D

Si se les entrega un arreglo de mayor dimensión este se aplanará automaticamente

In [ ]:
A = np.arange(6)
B = np.array([0, 1, 10, 100])
display(A, 
        B)

La unión e intersección, respectivamente:

In [ ]:
display(np.union1d(A, B),
        np.intersect1d(A, B))

O los elementos que existen a A y no en B (y viceversa)

In [ ]:
display(np.setdiff1d(A, B),
        np.setdiff1d(B, A))

### Ordenando arreglos

NumPy provee la función `np.sort` para ordernar un ndarray

Se puede usar el argumento `kind` para escoger distintos algoritmos de ordenamiento (por defecto quicksort)

El argumento `axis` especifica que eje se va a ordenar

In [ ]:
A = np.random.randn(2, 2)
display(A)
display(np.sort(A, axis=1))
display(np.sort(A, axis=0))
display(np.sort(A, axis=None))

La función `np.argsort` entrega un arreglo de índices que ordena el arreglo de menor a mayor

In [ ]:
A = np.array(["A", "B", "C"])
B = np.array([2, 4, 1])

idx = np.argsort(B)
display(idx)
display(A[idx])

## Tópicos extra: 

**NumPy para usuarios de Matlab**

En la documentación de NumPy, en la sección [NumPy para usuarios de Matlab](https://numpy.org/devdocs/user/numpy-for-matlab-users.html) se describen las diferencias clave entre NumPy y Matlab y se presenta una tabla de equivalencias entre las funciones de ambos


**Extender NumPy**

NumPy provee una [API en lenguaje C para manipular ndarray a bajo nivel](https://numpy.org/devdocs/user/c-info.html)

La API se puede usar para crear nuevas funcioens y módulos que utilicen ndarray

**Computación simbólica** 

La computación simbólica es un paradigma donde los cálculos se hacen de forma *análitica* en lugar de *númerica*

Se definen variables o simbolos que son operados algebraicamente

Este paradigma se usa tipicamente para obtener expresiones simplificadas de derivadas o integrales, series, límites, factorizaciones, expansiones, etc

- Paradigma númerico: Nos da el resultado de una expresión
- Paradigma simbólico: Nos da la expresión

En Python se puede hacer computación simbólica con [SimPy](https://www.sympy.org)
